Create a VGG model from scratch

In [1]:
import numpy as np
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image


Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


Model creation


In [2]:
def ConvBlock(layers, model, filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
        

In [3]:
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

Copy data pre-processing code

In [4]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

define vgg model architecture

In [5]:
def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))
    
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)
    
    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    # original classification problem over 1000 classes
    model.add(Dense(1000, activation='softmax'))
    return model

In [6]:
model = VGG_16()

In [7]:
FILES_PATH = 'http://www.platform.ai/models/'
fpath = get_file('vgg16.h5', FILES_PATH+'vgg16.h5', cache_subdir='models')
model.load_weights(fpath)

setup methods for grabbing the data

In [8]:
batch_size=64
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=batch_size, class_mode='categorical'):
    return gen.flow_from_directory(dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

grab the data

In [9]:
batches = get_batches('~/data/dogs-vs-cats-redux/train', batch_size=batch_size)
val_batches = get_batches('~/data/dogs-vs-cats-redux/valid', batch_size=batch_size)


OSError: [Errno 2] No such file or directory: '~/data/dogs-vs-cats-redux/train'

Manually fine-tune model by removing the last layer, setting all remaining layers as not trainable and then adding new final layer for binary classification

In [ ]:
model.pop()
for layer in model.layers:
    layer.trainable = False
# why specifying input shape in model.add here?
model.add(Dense(2, activation='softmax'))
# fixed learning rate here
model.compile(optimizer=RMSprop(lr=0.01),
              loss='categorical_crossentropy', metrics=['accuracy'])
# test different optimizer
# copied from: https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3
#sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
num_epoch=20
model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=num_epoch,
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

In [ ]:
%matplotlib inline
from __future__ import division,print_function
from matplotlib import pyplot as plt
# use provided utility functions
import utils; reload(utils)
from utils import plots

In [ ]:
# load some sample test data
test_batches = get_batches('~/data/dogs-vs-cats-redux/test', shuffle=False, batch_size=1)
fnames = test_batches.filenames
imgs,labels = next(test_batches)
plots(imgs, titles=fnames[:1])

In [ ]:
#plots(imgs, titles=labels)
print(len(imgs))

In [ ]:
model.predict(imgs)

In [ ]:
model.predict_proba(imgs)

In [ ]:
test_preds = model.predict(imgs)
test_idxs = np.argmax(test_preds, axis=1)
test_probs = [test_preds[i, test_idxs[i]] for i in range(len(test_idxs))]
print(test_probs)
print(test_idxs)

In [ ]:
type(test_batches).__name__

Save sample results for 12500 predictions to file

In [ ]:
fnames[:8]

run prediction on all
need to iterate over batches
write out results to file

In [ ]:
import csv

sbatch_size = 64
submit_batches = get_batches('~/data/dogs-vs-cats-redux/test', shuffle=False, batch_size=sbatch_size)
fnames = submit_batches.filenames

f = open('lesson1_submit_numepoch_20.csv', 'wt')
writer = csv.writer(f)
for i in xrange(0, len(fnames), sbatch_size):
    snames = fnames[i:i+sbatch_size]
    imgs,labels = next(submit_batches)
    spreds = model.predict(imgs)
    sidxs = np.argmax(spreds, axis=1)
    sprobs = [spreds[i, sidxs[i]] for i in range(len(sidxs))]
    sidxs
    #print(sprobs)
    for j in range(len(snames)):
        if sidxs[j] == 1:
            writer.writerow((snames[j].strip('unknown/').strip('.jpg'), sprobs[j]))
        else:
            writer.writerow((snames[j].strip('unknown/').strip('.jpg'), 1-sprobs[j]))
f.close()